In [20]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))


['data.npy', 'labels.npy']


In [21]:

from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import SeparableConv2D,Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

class anandnet:
    @staticmethod
    def build(width, height, depth, classes):
        
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1

        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1

        model.add(SeparableConv2D(32, (3, 3), padding="same",
            input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(32, (3, 3), padding="same",
            input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.10))

        model.add(SeparableConv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(SeparableConv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.30))


        model.add(SeparableConv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(SeparableConv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(256))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))


        model.add(Dense(classes))
        model.add(Activation("softmax"))

        return model

In [22]:
!pip install imutils
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
import tensorflow as tf
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os
from keras import backend as K

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)


EPOCHS = 32
INIT_LR = 1e-3 
BS = 32 

print("Load images' NPY file")
data = []
labels = []

random.seed(42)

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)
data=np.load('../input/data.npy')
labels=np.load('../input/labels.npy')

(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=6)

trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,horizontal_flip=True, fill_mode="nearest")

# initialize the model
print("Compiling model...")
model = anandnet.build(width=128, height=128, depth=3, classes=2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS) #Optimise uisng Adam 
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
    
# train the network
print("Training network")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX)//BS,
	epochs=EPOCHS, verbose=1,callbacks=[tensorboard])

label_name=["benign","malicious"]

print("Save the model for the applied CNN")

print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=128)
print(classification_report(testY.argmax(axis=1),
predictions.argmax(axis=1)))

cm = confusion_matrix(testY.argmax(axis=1), predictions.argmax(axis=1))
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))

plt.style.use("ggplot")
plt.figure()
N = EPOCHS

K.clear_session()

Load images' NPY file
Compiling model...
Training network
Epoch 1/32
52/52 [==============================] - 8s 160ms/step - loss: 0.6469 - acc: 0.7674 - val_loss: 0.5370 - val_acc: 0.7849
Epoch 2/32
52/52 [==============================] - 7s 125ms/step - loss: 0.5128 - acc: 0.7773 - val_loss: 0.5021 - val_acc: 0.7921
Epoch 3/32
52/52 [==============================] - 7s 129ms/step - loss: 0.4385 - acc: 0.7998 - val_loss: 0.6893 - val_acc: 0.7043
Epoch 4/32
52/52 [==============================] - 7s 128ms/step - loss: 0.4138 - acc: 0.8139 - val_loss: 0.4529 - val_acc: 0.7724
Epoch 5/32
52/52 [==============================] - 7s 129ms/step - loss: 0.4110 - acc: 0.8173 - val_loss: 0.6278 - val_acc: 0.7222
Epoch 6/32
52/52 [==============================] - 7s 137ms/step - loss: 0.4222 - acc: 0.8013 - val_loss: 1.1544 - val_acc: 0.7240
Epoch 7/32
52/52 [==============================] - 7s 141ms/step - loss: 0.3980 - acc: 0.8218 - val_loss: 0.7918 - val_acc: 0.6953
Epoch 8/32
52/52 [